In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lower,desc, mean, explode, lit, dayofmonth, to_date, cast
import pandas as pd
from pyspark.sql.types import *
from pyspark.sql import functions


spark = SparkSession.builder.master("local[*]").getOrCreate()

# SI TRABAJAMOS EN EL CLUSTER DE LA UNI NECESITAREMOS LA SIGUIENTE LINEA:
# spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")

<h1>Antes del confinamiento: desde 1/12/19 hasta 13/03/20</h1>

In [2]:
diciembre19 = spark.read.csv('12-2019.csv', header=True, inferSchema=True, sep=";")
enero20 = spark.read.csv('01-2020.csv', header=True, inferSchema=True, sep=";")
febrero20 = spark.read.csv('02-2020.csv', header=True, inferSchema=True, sep=";")
marzo20 = spark.read.csv('03-2020.csv', header=True, inferSchema=True, sep=";")

In [3]:
diciembre19 = diciembre19.filter(dayofmonth(to_date(col("fecha"),"yyyy-mm-dd"))>=1)
marzo20 = marzo20.filter(dayofmonth(to_date(col("fecha"),"yyyy-mm-dd"))<=13)

unionFase1 = diciembre19.union(enero20).union(febrero20).union(marzo20)
intFase1 = unionFase1.groupBy("id").mean("intensidad")
intFase1 = intFase1.withColumn("Fases", lit('Pre Covid-19'))
intFase1 = intFase1.withColumnRenamed("avg(intensidad)", "intensidad")
intFase1 = intFase1.drop("id").groupBy("Fases").mean("intensidad")

<h1>Primer estado de alarma: desde 14/03/20 hasta 20/06/20</h1>

In [5]:
marzo20 = spark.read.csv('03-2020.csv', header=True, inferSchema=True, sep=";")
abril20 = spark.read.csv('04-2020.csv', header=True, inferSchema=True, sep=";")
mayo20 = spark.read.csv('05-2020.csv', header=True, inferSchema=True, sep=";")
junio20 = spark.read.csv('06-2020.csv', header=True, inferSchema=True, sep=";")


In [6]:
marzo20 = marzo20.filter(dayofmonth(to_date(col("fecha"),"yyyy-mm-dd"))>=14)
junio20 = junio20.filter(dayofmonth(to_date(col("fecha"),"yyyy-mm-dd"))<=20)

unionFase2 = marzo20.union(abril20).union(mayo20)
intFase2 = unionFase2.groupBy("id").mean("intensidad")
intFase2 = intFase2.withColumn("Fases", lit('Primer estado'))
intFase2 = intFase2.withColumnRenamed("avg(intensidad)", "intensidad")
intFase2 = intFase2.drop("id").groupBy("Fases").mean("intensidad")

In [7]:
intFase2.show()

+-------------+-----------------+
|        Fases|  avg(intensidad)|
+-------------+-----------------+
|Primer estado|136.3331458375607|
+-------------+-----------------+



<h1>Nueva normalidad: desde 21/06/20 hasta 20/10/20</h1>

In [8]:
junio20 = spark.read.csv('06-2020.csv', header=True, inferSchema=True, sep=";")
julio20 = spark.read.csv('07-2020.csv', header=True, inferSchema=True, sep=";")
agosto20 = spark.read.csv('08-2020.csv', header=True, inferSchema=True, sep=";")
septiembre20 = spark.read.csv('09-2020.csv', header=True, inferSchema=True, sep=";")
octubre20 = spark.read.csv('10-2020.csv', header=True, inferSchema=True, sep=";")

In [9]:
mayo20 = mayo20.filter(dayofmonth(to_date(col("fecha"),"yyyy-mm-dd"))>=21)
octubre20 = octubre20.filter(dayofmonth(to_date(col("fecha"),"yyyy-mm-dd"))<=20)

unionFase3 = marzo20.union(junio20).union(mayo20).union(julio20).union(agosto20).union(septiembre20).union(octubre20)
intFase3 = unionFase3.groupBy("id").mean("intensidad")
intFase3 = intFase3.withColumn("Fases", lit('Nueva normalidad'))
intFase3 = intFase3.withColumnRenamed("avg(intensidad)", "intensidad")
intFase3 = intFase3.drop("id").groupBy("Fases").mean("intensidad")

In [10]:
intFase3.show()

+----------------+-----------------+
|           Fases|  avg(intensidad)|
+----------------+-----------------+
|Nueva normalidad|288.1112356542706|
+----------------+-----------------+



<h1>Segundo estado de alarma: desde 21/10/20 hasta 9/05/21</h1>

In [11]:
octubre20 = spark.read.csv('10-2020.csv', header=True, inferSchema=True, sep=";")
noviembre20 = spark.read.csv('11-2020.csv', header=True, inferSchema=True, sep=";")
diciembre20 = spark.read.csv('12-2020.csv', header=True, inferSchema=True, sep=";")
enero21 = spark.read.csv('01-2021.csv', header=True, inferSchema=True, sep=";")
febrero21 = spark.read.csv('02-2021.csv', header=True, inferSchema=True, sep=";")
marzo21 = spark.read.csv('03-2021.csv', header=True, inferSchema=True, sep=";")
abril21 = spark.read.csv('04-2021.csv', header=True, inferSchema=True, sep=";")
mayo21 = spark.read.csv('05-2021.csv', header=True, inferSchema=True, sep=";")

In [12]:
octubre20 = octubre20.filter(dayofmonth(to_date(col("fecha"),"yyyy-mm-dd"))>=21)
mayo21 = mayo21.filter(dayofmonth(to_date(col("fecha"),"yyyy-mm-dd"))<=9)

unionFase4 = octubre20.union(junio20).union(noviembre20).union(diciembre20).union(enero21).union(febrero21).union(marzo21).union(abril21).union(mayo21)
intFase4 = unionFase4.groupBy("id").mean("intensidad")
intFase4 = intFase4.withColumn("Fases", lit('Segundo estado'))
intFase4 = intFase4.withColumnRenamed("avg(intensidad)", "intensidad")
intFase4 = intFase4.drop("id").groupBy("Fases").mean("intensidad")

In [13]:
intFase4.show()

+--------------+-----------------+
|         Fases|  avg(intensidad)|
+--------------+-----------------+
|Segundo estado|335.3101337644425|
+--------------+-----------------+



<h1>Después de segundo estado de alarma: desde 9/05/21 hasta 1/11/21</h1>

In [14]:
mayo21 = spark.read.csv('05-2021.csv', header=True, inferSchema=True, sep=";")
junio21 = spark.read.csv('06-2021.csv', header=True, inferSchema=True, sep=";")
julio21 = spark.read.csv('07-2021.csv', header=True, inferSchema=True, sep=";")
agosto21 = spark.read.csv('08-2021.csv', header=True, inferSchema=True, sep=";")
septiembre21 = spark.read.csv('09-2021.csv', header=True, inferSchema=True, sep=";")
octubre21 = spark.read.csv('10-2021.csv', header=True, inferSchema=True, sep=";")
noviembre21 = spark.read.csv('11-2021.csv', header=True, inferSchema=True, sep=";")
# diciembre21 = spark.read.csv('/data/trafico/DataTrafico/2021/12-2021.csv', header=True, inferSchema=True, sep=";")

In [15]:
mayo21 = mayo21.filter(dayofmonth(to_date(col("fecha"),"yyyy-mm-dd"))>=9)
noviembre21 = noviembre21.filter(dayofmonth(to_date(col("fecha"),"yyyy-mm-dd"))<=1)

unionFase5 = octubre20.union(mayo21).union(junio21).union(julio21).union(agosto21).union(septiembre21).union(octubre21).union(noviembre21)
intFase5 = unionFase5.groupBy("id").mean("intensidad")
intFase5 = intFase5.withColumn("Fases", lit('Pos-segundo estado'))
intFase5 = intFase5.withColumnRenamed("avg(intensidad)", "intensidad")
intFase5 = intFase5.drop("id").groupBy("Fases").mean("intensidad")

In [16]:
intFase5.show()

+------------------+-----------------+
|             Fases|  avg(intensidad)|
+------------------+-----------------+
|Pos-segundo estado|350.9154890837606|
+------------------+-----------------+



In [17]:
unionFinal = intFase1.union(intFase2).union(intFase3).union(intFase4).union(intFase5)
nombre = unionFinal.withColumnRenamed("avg(intensidad)", "Intensidad media de tráfico ")
union = nombre.toPandas()
print(union)

                Fases  Intensidad media de tráfico 
0        Pre Covid-19                    419.733556
1       Primer estado                    136.333146
2    Nueva normalidad                    288.111236
3      Segundo estado                    335.310134
4  Pos-segundo estado                    350.915489
